Preprocessing - **SparsePCA** (component=1)  
Algorithm - **Sparse Classification**  
Feature - **Real + Imaginary**


In [1]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import SparsePCA

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

n_comp = 20
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

measurement = np.load('../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:3000]
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

In [2]:
# ---------------------------------------- Preprocessing ----------------------------------------
def getRealImaginaryParts(cirs):
    real = cirs[:, :, 0]
    imag = cirs[:, :, 1]
    return real, imag

def apply_sparse_pca(data, n_components):

    # data : (1600, 251, 2)
    reshaped_data = data.reshape(data.shape[0], -1) # (1600, 502)
    
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(reshaped_data)
    
    spca = SparsePCA(n_components=n_components, random_state=42)
    data_spca = spca.fit_transform(data_scaled) # (1600, 2)
    
    return data_spca, scaler, spca

# -------------- Training ----------------
# Feature Extraction
train_alice_cirs = trainCIR[:, alice_channel, :, :]  
train_eve_cirs = trainCIR[:, eve_channel, :, :]
train_cirs = np.vstack((train_alice_cirs, train_eve_cirs)) 
train_cirs_spca, scaler, spca = apply_sparse_pca(train_cirs, n_components=n_comp)

# Labels
train_alice_labels = np.zeros(train_alice_cirs.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_cirs.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Atoms
train_atoms = train_cirs_spca

# Dictionary 
D = train_atoms.T
print(f'Dictionary : {D.shape}')


# -------------- Testing ----------------
# Feature Extraction
test_alice_cirs = testCIR[:, alice_channel, :, :]
test_eve_cirs = testCIR[:, eve_channel, :, :]
test_cirs = np.vstack((test_alice_cirs, test_eve_cirs))  # (samples, 251, 2)

# Reshape
reshaped_test_cirs = test_cirs.reshape(test_cirs.shape[0], -1)  # (samples, 251*2)
# Scaling
test_cirs_scaled = scaler.transform(reshaped_test_cirs)
# Sparse PCA
test_cirs_spca = spca.transform(test_cirs_scaled)  # (samples, n_components)

# Labels
alice_test_labels = np.zeros(test_alice_cirs.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(test_eve_cirs.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((alice_test_labels, eve_test_labels))

# test Atoms
test_atoms = test_cirs_spca

KeyboardInterrupt: 

In [ ]:
# ---------------------------------------- Sparse Classification ----------------------------------------
# Sparse Coding Function

# Sparse Coefficients
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=10):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Calculate Residuals for Each Class
def calculate_residual(tSample, coefficients, class_indices, D):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep only coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Classification Function
def classify_signal(tSample, D, trainLabel):
    
    coefficients = find_sparse_coefficients(tSample, D)
    
    residuals = []

    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, coefficients, class_indices, D)
        residuals.append(residual)

    min_residual_index = np.argmin(residuals)
    predicted_class = unique_classes[min_residual_index]
    
    return predicted_class

predictions = []

# Classifying Test Data
for testSample in test_atoms:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)

In [ ]:
# ---------------------------------------- Evaluation ----------------------------------------
accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
print(f"\nTotal testing channel: {test_labels.shape}")

tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# Gamma calculation
gamma = (tp + fn) / (tn + fp)

# Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

Classification Accuracy: 56.00%

Total testing channel: (1200,)
tp: 386
tn: 286
fp: 314
fn: 214
MDR: 0.5233333333333333
FAR: 0.3566666666666667
AR: 0.56
